<h1 style="text-align:center;">Setting the IP configuration for the Manager ethernet interface</h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>The objective of this script is to demonstrate how to <b>set the static IP configuration for the Manager ethernet interface</b> of an HPE iLO system using the Redfish API.<br></p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
from get_resource_directory import get_resource_directory

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <div style = "color:green; font-size:40px;">2.</div><br>
    <p>The following code defines the set_ilo_static_ipv4 function. This function is used to set the static IPv4 configuration for the Manager ethernet interface in an iLO (Integrated Lights-Out) server.</p><br>

1. Retrieve the resource directory to find the URI for the ethernet interfaces or manually search for the relevant URI if the resource directory is disabled.
2. Retrieve the current configuration of the ethernet interfaces, including DHCPv4 settings, IPv4 addresses, and static name servers, and display them.
3. Iterate through each ethernet interface and prompt the user to choose which interface to modify.
4. Disable the UseDNSServers and UseGateway options if they are enabled in the DHCPv4 settings.
5. Update the IPv4Addresses and StaticNameServers with the provided values in the function parameters, or create a new body with the DNS servers if StaticNameServers are not available.
6. Send a PATCH request to update the ethernet interface configuration with the new settings.
Call the ilo_response function to handle the response and display any error messages or the updated configuration.
7. Exit the loop after processing the first ethernet interface.<br><br>
Overall, the function retrieves the current ethernet interface configuration, allows the user to choose which interface to modify, and updates the configuration with the provided IPv4 and DNS settings.
</div>

In [ ]:
def set_ilo_static_ipv4(_redfishobj, ipv4_dict, dns_dict):

    ethernet_data = {}
    body = dict()

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        manager_ethernet_interfaces = managers_members_response.obj['EthernetInterfaces']\
                                                                                    ['@odata.id']
        manager_ethernet_interfaces_response = _redfishobj.get(manager_ethernet_interfaces)
        manager_ethernet_interfaces_members = manager_ethernet_interfaces_response.\
                                                            obj['Members']
        for _member in manager_ethernet_interfaces_members:
            _tmp = _redfishobj.get(_member['@odata.id']).obj
            ethernet_data[_member['@odata.id']] = _tmp
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#EthernetInterfaceCollection.' in instance['@odata.type'] and 'Managers' in \
                                                                        instance['@odata.id']:
                ethernet_uri = instance['@odata.id']
                ethernet_interfaces = _redfishobj.get(ethernet_uri).obj['Members']
                for _ethernet_interface in ethernet_interfaces:
                    ethernet_data[_ethernet_interface['@odata.id']] = _redfishobj.\
                                                        get(_ethernet_interface['@odata.id']).dict
                break

    if ethernet_data:
        print("\n\nShowing all available ethernet management interfaces before changes:\n\n")
        for interface in ethernet_data:
            sys.stdout.write("Ethernet Management Inteface \'%s\'\n" % ethernet_data\
                                                                            [interface].get('Id'))

            sys.stdout.write("\'DHCPv4\':\n")
            if ethernet_data[interface].get('DHCPv4'):
                print(json.dumps(ethernet_data[interface].get('DHCPv4'), indent=4, sort_keys=True))
            else:
                print(json.dumps(ethernet_data[interface]['Oem']['Hpe'].get('DHCPv4'), indent=4, \
                                                                                sort_keys=True))
            sys.stdout.write("\'IPv4\':\n")
            if ethernet_data[interface].get('IPv4StaticAddresses'):
                print(json.dumps(ethernet_data[interface].get('IPv4Addresses'), indent=4, \
                                                                                sort_keys=True))
            sys.stdout.write("\'StaticNameServers\':\n")
            if ethernet_data[interface].get('StaticNameServers'):
                print(json.dumps(ethernet_data[interface].get('StaticNameServers'), indent=4, \
                                                                                sort_keys=True))

    for ethernet in ethernet_data:
        sys.stdout.write("Ethernet Interface: %s\n" % ethernet)
        ans = input("Would you like to modify this interface? (y/n)\n")
        if "n" in ans:
            continue
        if 'DHCPv4' in ethernet_data[ethernet]:
            if ethernet_data[ethernet]['DHCPv4'].get('UseDNSServers'):
                resp = _redfishobj.patch(ethernet, {"DHCPv4": {"UseDNSServers": False}})
                ilo_response(_redfishobj, resp)
            if ethernet_data[ethernet]['DHCPv4'].get('UseGateway'):
                resp = _redfishobj.pathc(ethernet, {"DHCPv4": {"UseGateway": False}})
                ilo_response(_redfishobj, resp)
        if 'IPv4StaticAddresses' in ethernet_data[ethernet]:
            body.update({"IPv4Addresses": [ipv4_dict]})
        if 'StaticNameServers' in ethernet_data[ethernet]:
            body.update({"StaticNameServers" : [dns_dict.get('PrimaryDNS'), \
                                                                    dns_dict.get('SecondaryDNS')]})
        else:
            body.update({"Oem": {"Hpe": {"IPv4": {"DNSServers": [dns_dict.get('PrimaryDNS'), \
                                                                 dns_dict.get('SecondaryDNS')]}}}})
        resp = _redfishobj.patch(ethernet, body)
        ilo_response(_redfishobj, resp)
        break


<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <div style = "color:green; font-size:40px;">3.</div><br>
    <p>The following code defines the ilo_response function. This function is used to handle the response from an iLO (Integrated Lights-Out) server after sending a PATCH request to update the configuration.</p><br>
1. Check if the response status is 400. If it is, try to access the extended error message from the response object and print it in a formatted JSON representation.
2. If an exception occurs while accessing the extended error message, print a generic error message indicating that the extended message info cannot be accessed.
3. If the response status is not 400 (indicating an error), write a stderr message indicating the HTTP response status.
4. If the response status is 200 (indicating success), print a success message suggesting to reset iLO for the settings to take effect.
5. Print the response dictionary in a formatted JSON representation.
<br><br>
Overall, the function retrieves the current ethernet interface configuration, allows the user to choose which interface to modify, and updates the configuration with the provided IPv4 and DNS settings.
</div>

In [ ]:
def ilo_response(_redfishobj, resp):
    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                            sort_keys=True))
        except Exception as excp:
            sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                             "Message Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
    else:
        print("Success! Suggest to reset iLO for settings to take effect.\n")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <div style = "color:green; font-size:40px;">4.</div><br>
    
Next is the main function. The code establishes a connection with a target server using Redfish, a standardized API for server management. It sets the URL, login credentials, and defines dictionaries for IPv4 and DNS settings. It creates a Redfish client object and attempts to log in using the provided credentials. If successful, it calls the set_ilo_static_ipv4 function to configure the IPv4 and DNS settings for a specific Ethernet interface on the server. The function retrieves the necessary information, displays the current settings, and prompts the user to choose an interface for modification. If selected, it updates the DHCPv4 and IPv4 static address settings based on the provided dictionaries and sends the changes to the server. Finally, the code logs out of the Redfish session.
<br><br>
In summary, the code establishes a connection with a server, configures network settings using Redfish, and performs the necessary operations to modify the IPv4 and DNS settings for a specific Ethernet interface on the server.
</div>

<div class = "alert alert-block alert-warning" style="border-radius: 20px;">
    <b>When running on the server locally</b> use the following values<br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    #IPv4 settings for Address, Gateway and SubnetMask as well as DNS.
    IPV4_DICT = {'Address':'16.83.61.123', \
                 'Gateway':'16.83.60.1', \
                 'SubnetMask':'255.255.252.0' \
                 }
    DNS_DICT = {'PrimaryDNS':'16.110.135.52', \
                'SecondaryDNS':'16.110.135.51' \
                }

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    set_ilo_static_ipv4(REDFISHOBJ, IPV4_DICT, DNS_DICT)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.REST API: <a href = "https://restfulapi.net/">LINK</a><br>
    4.Changing Boot Setting Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/?python#changing-boot-settings">LINK</a><br>
    
</div>